In [1]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math as mt

In [2]:
iris = datasets.load_iris()

In [3]:
iris.target_names, iris.feature_names

(array(['setosa', 'versicolor', 'virginica'], dtype='<U10'),
 ['sepal length (cm)',
  'sepal width (cm)',
  'petal length (cm)',
  'petal width (cm)'])

In [4]:
df = pd.DataFrame(iris.data)
df.columns = iris.feature_names
df['flower_type']=iris.target
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),flower_type
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [10]:
''' max_gain_splitter function is used to calculate
    gain ratio(may be max_gain_ratio) of feature "f_name" from mid point ("boundary")
    
    Parameters
    ----------
    x: all features
    y: target
    f_name: a feature name for which gain ratio is to be calculated
    before_split: values of all different classes('setosa'=0, 'versicolor'=1, 'virginica'=2) before splitting
    p_entropy: entropy of parent node
    boundary: mid point of two continous value of feature f_name
    
    Returns
    -------
    gain_ratio: gain ratio of feature "f_name" from mid point ("boundary") 
    
'''

def max_gain_splitter(x, y, f_name, before_split, p_entropy, boundary):
    total_len = len(y) # or len(x)
    unq_classes = len(before_split) # total number of unique output or unique classes
    after_split_left = [0] * unq_classes
    
    ''' Calculating count of each classes in child node (or after split classes count) '''
    for i in range(unq_classes):
        after_split_left[i] = (y[x[f_name] <= boundary] == i).sum()
    
    ''' Calculating length of left and right child node (or total count of classes) '''
    left_len = sum(after_split_left)
    right_len = total_len - left_len
    
    l_entropy = r_entropy = 0
    
    ''' Calculating Entropy of left child '''
    for i in range(unq_classes):
        if after_split_left[i]: # to save from division of zero error (in log2)
            l_entropy += (after_split_left[i] / left_len) * (mt.log2( left_len / after_split_left[i] ))
    l_entropy *= (left_len / total_len) # for weightage addition
            
    ''' Calculating Entropy of right child '''
    for i in range(unq_classes):
        if before_split[i] > after_split_left[i]: # to save from division of zero error (in log2)
            r_entropy += ((before_split[i] - after_split_left[i]) / right_len) * (mt.log2( right_len / ( before_split[i] - after_split_left[i] )))
    r_entropy *= (right_len / total_len) # for weightage addition
    
    ''' Calculating Gain Ratio '''
    gain_ratio = (p_entropy - l_entropy - r_entropy) # information gain
    # division by split info
    gain_ratio /= (left_len / total_len) * mt.log2( total_len / left_len ) + (right_len / total_len) * mt.log2( total_len / right_len )
    
    return gain_ratio


''' splitter function is used to calculate max_gain_ratio of particular feature f_name.
    splitter function is calculate gain ratio for every middle point of two continuous
    value of feature f_name and then by comparing finding the max_gain ratio among all
    the values.
    to find gain ratio at every middle point splitter function calls the "max_gain_splitter" function.
    
    Parameter
    ---------
    x: all features
    y: target
    f_name: particular feature name
    
    Returns
    -------
    max_gain_ratio: max_gain_ratio for feature f_name
    p_entropy: entropy of parent node
    final_mid_point: mid point for which gain ratio is max for feature f_name
    before_split: values of all different classes('setosa'=0, 'versicolor'=1, 'virginica'=2) before splitting
'''

def splitter(x, y, f_name):
    continuous_x = sorted(list(x[f_name].unique())) # finding all unique continous value of f_name and arranging them in increasing order  
    total_len = len(y)
    unq_classes = 3
    before_split = [0] * unq_classes
    
    ''' Calculating count of each classes in parent node (or before_split classes count) '''
    for i in range(unq_classes):
        before_split[i] = (y==i).sum()
    
    ''' Calculating entropy of current node (or parent node) '''
    p_entropy = 0 
    for i in range(unq_classes):
        if before_split[i]: # to save from division of zero error (in log2)
            p_entropy += (before_split[i] / total_len) * mt.log2( total_len / before_split[i] )
    
    max_gain_ratio  = -2**31
    final_mid_point = 0
    for i in range(len(continuous_x) - 1):
        mid = (continuous_x[i] + continuous_x[i+1]) / 2 # finding middle point of every value of feature f_name
        temp_gain_ratio = max_gain_splitter(x, y, f_name, before_split, p_entropy, mid)
        if temp_gain_ratio > max_gain_ratio:
            max_gain_ratio = temp_gain_ratio
            final_mid_point = mid
    return max_gain_ratio, p_entropy, final_mid_point, before_split
    
    
''' print_splits_info function is used to print the required output
    and finding the best feature for splitting(having maximum gain ratio)
    
    Parameters
    ----------
    x: all features
    y: target
    f_names: all feature names list
    lvl: level of tree's nodes
'''

def print_splits_info(x, y, f_names, lvl):
    p_entropy = final_mid_point = 0
    before_split = []
    max_gain_ratio = -2**31
    
    ''' FINDING BEST FEATURE FOR SPLIT '''
    i = j = 0
    f_len = len(f_names)    
    while j < f_len:
        temp_gain, p_entropy, mid_temp, before_split = splitter(x, y, f_names[j])
        if temp_gain > max_gain_ratio:
            max_gain_ratio = temp_gain
            final_mid_point = mid_temp
            i=j
        j += 1
    
    ''' Checking if current node is leaf node or internal node
        by checking if there is only one class after splitting
    '''
    flag = True
    chk = sum(before_split)
    for k in before_split :
        if k == chk:
            flag = False
            break
            
    ''' Printing Solution '''
    print('Level ', lvl)
    for k in range(len(before_split)):
        if before_split[k]:
            print('Count of', k, before_split[k])
    print('Current Entropy is = ', p_entropy)
    if not flag and chk:
        print('Reached leaf Node')
    else:
        print('Splitting on feature <', f_names[i],  '> with gain ratio', max_gain_ratio)
    print()
    
    ''' Splitting on feature f_names[i] and calling recursively'''
    if flag and chk:
        x1, y1 = x[x[f_names[i]] <= final_mid_point], y[x[f_names[i]] <= final_mid_point]
        x2, y2 = x[x[f_names[i]] > final_mid_point], y[x[f_names[i]] > final_mid_point]
        print_splits_info(x1, y1, (f_names), lvl + 1)
        print_splits_info(x2, y2, (f_names), lvl + 1)
        

In [11]:
x = df.iloc[:, :-1]
y = df['flower_type']
f_names = list(df.columns[:-1])
print_splits_info(x, y, f_names, 0) # start from level zero

Level  0
Count of 0 50
Count of 1 50
Count of 2 50
Current Entropy is =  1.5849625007211559
Splitting on feature < petal length (cm) > with gain ratio 0.9999999999999999

Level  1
Count of 0 50
Current Entropy is =  0.0
Reached leaf Node

Level  1
Count of 1 50
Count of 2 50
Current Entropy is =  1.0
Splitting on feature < petal width (cm) > with gain ratio 0.6933647985912663

Level  2
Count of 1 49
Count of 2 5
Current Entropy is =  0.44506485705083854
Splitting on feature < petal length (cm) > with gain ratio 0.6066178220203001

Level  3
Count of 1 49
Count of 2 3
Current Entropy is =  0.3182152976832332
Splitting on feature < petal length (cm) > with gain ratio 0.2720453440631931

Level  4
Count of 1 47
Count of 2 1
Current Entropy is =  0.14609425012013613
Splitting on feature < petal width (cm) > with gain ratio 1.0

Level  5
Count of 1 47
Current Entropy is =  0.0
Reached leaf Node

Level  5
Count of 2 1
Current Entropy is =  0.0
Reached leaf Node

Level  4
Count of 1 2
Count of 

## IRIS_DATASET CLASSIFIER TREE (USING export_graphviz)

<img src="iris_tree.png">

## IRIS_DATASET CLASSIFIER TREE (MANUALLY)

<img src="iris_tree_manual.png">